In this notebook, I have trained an autoencoder to mitigate noisy images form the MNIST dataset. The Autoencoder consists of two components :-

(1) Enoder - which ecnodes data onto some latent space, that usually has significantly less dimensionality than the original data.

(2) Decoder - which tries to reconstruct the original data back from its encoded latent space representation.

So, essentially the latent representation serves as a bottleneck, that can be utilized for a wide range of tasks. The common ones include data compression, dimensionality reduction, new data generation etc.

To prevent the network from simply learning some form of identity mapping, adding certain amount of noise to the input data became popular practice. After a while, it was realized that autoencoders could in fact be used for de-noising data applications alone.

I imported the the MNIST dataset, added some noise to it, and then trained an autoencoder on it which uses Convolution layers for encoding and decoding part. The images are of thge shape (28, 28), and the latent space is a vector of length (16). The code is pretty straighforward, and easy to follow. At least, I hope so. This is a simple example, but it can be used as a template for de-noising more sophisticated data.

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose 
from tensorflow.keras.layers import LeakyReLU 
from tensorflow.keras.layers import Activation 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Reshape 
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def ConvAutoEncoder(width, height, depth, filters=(32, 64), latentDim=16):

  input_shape = (width, height, depth)
  chanDim = -1

  encInp = Input(shape=input_shape)
  x = encInp

  for f in filters:
    x = Conv2D(f, (3, 3), strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(axis=chanDim)(x)

  volumeSize = K.int_shape(x)
  x = Flatten()(x)
  latent = Dense(latentDim)(x)

  encoder = Model(encInp, latent, name='encoder')

  decInp = Input(shape=(latentDim, ))
  x = Dense(np.prod(volumeSize[1:]))(decInp)
  x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

  for f in filters[::-1]:
    x = Conv2DTranspose(f, (3, 3), strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(axis=chanDim)(x)

  x = Conv2DTranspose(depth, (3, 3), padding='same')(x)
  output = Activation('sigmoid')(x)

  decoder = Model(decInp, output, name='decoder')

  autoEncoder = Model(encInp, decoder(encoder(encInp)), name='autoencoder')

  return (encoder, decoder, autoEncoder)

In [ ]:
data = mnist.load_data()
(trainX, _), (testX, _) = mnist.load_data()

trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

In [ ]:
trainNoise = np.random.uniform(low=-800, high=800, size=trainX.shape)/1000.0
testNoise = np.random.uniform(low=-800, high=800, size=testX.shape)/1000.0
trainXNoisy = np.clip(trainX + trainNoise, 0, 1)
testXNoisy = np.clip(testX + testNoise, 0, 1)

In [ ]:
test_set_len = len(testXNoisy)
split_at = int(0.7*test_set_len)

validXNoisy = testXNoisy[:split_at]
testXNoisy = testXNoisy[split_at:]

validX = testX[:split_at]
testX = testX[split_at:]

In [ ]:
(encoder, decoder, autoencoder) = ConvAutoEncoder(28, 28, 1)

In [ ]:
plot_model(autoencoder)

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [ ]:
opt = Adam(lr=1e-3)
autoencoder.compile(loss="mse", optimizer=opt)

In [ ]:
EPOCHS = 25
BS = 32

H = autoencoder.fit(
    trainXNoisy, trainX,
    validation_data=(validXNoisy, validX),
    epochs=EPOCHS,
    batch_size=BS)

In [ ]:
fontdict = {'weight':'bold', 'fontsize':14}
plt.figure(figsize=(8, 5))
plt.plot(H.history['loss'], label='trainLoss', linewidth=2)
plt.plot(H.history['val_loss'], label='valLoss', linewidth=2)
plt.xlabel('Epoch #', fontdict=fontdict )
plt.ylabel('Loss #', fontdict=fontdict )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
def plot_samples(n_samples, noisy_data):

  total_samples = len(noisy_data)
  sample_inds = np.random.choice([i for i in range(total_samples)], n_samples)

  y_hat = autoencoder.predict(noisy_data[sample_inds]).reshape(-1, 28, 28)
  y = noisy_data[sample_inds].reshape(-1, 28, 28)

  fig, ax = plt.subplots(n_samples, 2, figsize=(1.95,n_samples), gridspec_kw = {'wspace':0, 'hspace':0})
  for i in range(n_samples):
    ax[i, 0].imshow(y[i], 'gray')
    ax[i, 1].imshow(y_hat[i], 'gray')
    ax[i, 0].axis('off')
    ax[i, 1].axis('off')

In [ ]:
plot_samples(8, testXNoisy)